# screener

In [1]:
import os
os.chdir('/Users/siegstedt/projects/trading/binance')
!pwd

/Users/siegstedt/projects/trading/binance


In [2]:
import requests
import json
import pandas as pd
import numpy as np
import datetime as dt

#### request all tickers

In [4]:
import src.binance_helpers as help_to

In [5]:
all_tickers = help_to.get_all_tickers(markets=['USDT','EUR'])
all_tickers.head()

,symbol,price
0,BTCUSDT,62190.00000000
1,ETHUSDT,2461.03000000
2,BNBUSDT,541.98020000
3,BCCUSDT,448.70000000
4,NEOUSDT,100.11700000


In [29]:
# extract list of symbols
symbols = all_tickers.symbol.to_list()

In [30]:
# clean tickers from 'UP' 'DOWN' products
symbols = [
    i for i 
    in symbols
    if not ('UP' in i or 'DOWN' in i)
]

In [31]:
len(symbols)

281

#### request short-term data for symbol

In [ ]:
# check growth in volume
# check growth in closing price
# check for market cap
# create a short list
# sort list
# attach a prio ranking
# export data

In [32]:
# choose a symbol
symbol = symbols[0]
symbol

'BTCUSDT'

In [17]:
# get data for last hour in 5 minute intervals
end_time = dt.datetime.now()
start_time = dt.datetime.now() - dt.timedelta(seconds=24*60*60)

# request data
history = help_to.get_historical_bars(symbol, '5m', start_time, end_time)

In [33]:
history

,datetime,open,high,low,close,volume
2021-04-16 11:30:00,1618565400000,60726.26,60836.69,60510.00,60536.53,492.105996
2021-04-16 11:35:00,1618565700000,60536.53,60676.49,60400.00,60600.78,676.261717
2021-04-16 11:40:00,1618566000000,60600.79,60679.73,60460.00,60473.75,308.215551
2021-04-16 11:45:00,1618566300000,60481.03,60676.47,60416.66,60601.61,346.934147
2021-04-16 11:50:00,1618566600000,60601.62,60786.20,60483.77,60753.71,309.615667
...,...,...,...,...,...,...
2021-04-17 11:05:00,1618650300000,62311.15,62311.65,62180.00,62219.84,176.368244
2021-04-17 11:10:00,1618650600000,62219.83,62262.00,62136.68,62185.04,365.439783
2021-04-17 11:15:00,1618650900000,62185.04,62185.04,61957.52,62002.48,322.056382
2021-04-17 11:20:00,1618651200000,62002.49,62004.97,61779.51,61847.03,296.096622


In [75]:
# check for surge in volume in the last 30 minutes
history['volume_change_abs'] = history.volume - history.volume.shift(1)
history['volume_change_rel'] = history.volume_change_abs / history.volume.shift(1)

def _report_surge(volume_change_rel_series, mins, threshold=0.5):
    mins = int(mins/5)
    max_surge = volume_change_rel_series[-mins:].max()
    if max_surge > threshold:
        return True
    else:
        return False

# store values in dict
volume_surge_dict = {
    'volume_surge_15min': _report_surge(history['volume_change_rel'], 15, threshold=0.5),
    'volume_surge_30min': _report_surge(history['volume_change_rel'], 30, threshold=0.5),
    'volume_surge_60min': _report_surge(history['volume_change_rel'], 60, threshold=0.5),
}
volume_surge_dict

{'volume_surge_15min': False,
 'volume_surge_30min': True,
 'volume_surge_60min': True}

In [73]:
# check growth in closing price
history['close_change_abs'] = history.close - history.close.shift(1)
history['close_change_rel'] = history.close_change_abs / history.close.shift(1)

In [74]:
history

,datetime,open,high,low,close,volume,volume_change_abs,volume_change_rel,close_change_abs,close_change_rel
2021-04-16 11:30:00,1618565400000,60726.26,60836.69,60510.00,60536.53,492.105996,NaN,NaN,NaN,NaN
2021-04-16 11:35:00,1618565700000,60536.53,60676.49,60400.00,60600.78,676.261717,184.155721,0.374220,64.25,0.001061
2021-04-16 11:40:00,1618566000000,60600.79,60679.73,60460.00,60473.75,308.215551,-368.046166,-0.544236,-127.03,-0.002096
2021-04-16 11:45:00,1618566300000,60481.03,60676.47,60416.66,60601.61,346.934147,38.718596,0.125622,127.86,0.002114
2021-04-16 11:50:00,1618566600000,60601.62,60786.20,60483.77,60753.71,309.615667,-37.318480,-0.107566,152.10,0.002510
...,...,...,...,...,...,...,...,...,...,...
2021-04-17 11:05:00,1618650300000,62311.15,62311.65,62180.00,62219.84,176.368244,-32.196906,-0.154373,-91.31,-0.001465
2021-04-17 11:10:00,1618650600000,62219.83,62262.00,62136.68,62185.04,365.439783,189.071539,1.072027,-34.80,-0.000559
2021-04-17 11:15:00,1618650900000,62185.04,62185.04,61957.52,62002.48,322.056382,-43.383401,-0.118716,-182.56,-0.002936
2021-04-17 11:20:00,1618651200000,62002.49,62004.97,61779.51,61847.03,296.096622,-25.959760,-0.080606,-155.45,-0.002507


In [76]:
# store values in dict
close_surge_dict = {
    'close_surge_15min': _report_surge(history['close_change_rel'], 15, threshold=0.5),
    'close_surge_30min': _report_surge(history['close_change_rel'], 30, threshold=0.5),
    'close_surge_60min': _report_surge(history['close_change_rel'], 60, threshold=0.5),
}
close_surge_dict

{'close_surge_15min': False,
 'close_surge_30min': False,
 'close_surge_60min': False}